Prova 2º Bimestre

Nome William Douglas Costa Gomes

RA 48467

Questão 1 (2,5) - Raspando o site vivareal.com.br para obter os dados

Importação das bibliotecas

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
tamanho = []
quartos = []
banheiros = []
vagas = []
bairros = []
valor = []

In [11]:
for i in range(100):
  index = i + 1
  html=requests.get("https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/?pagina=" + str(index)).text

  soup=BeautifulSoup(html,'html5lib')

  div = soup.find('div', {'class' : 'results-main__panel js-list'})

  segunda_div = div.find('div', {'class' : 'results-list js-results-list'})

  terceiro_div = div.find_all('div', {'class' : 'js-card-selector'})

  informacoesAP = []
  for item in terceiro_div:
    # Localizando o preço
    valores = item.find('div', {'class' : 'property-card__price js-property-card-prices js-property-card__price-small'})
    try:
      valorEncontrado = valores.find('p').text.strip().split()
      valor.append(float(valorEncontrado[1]))
    except:
      continue

    # Localizando o nome dos bairros
    bairrosInformacoes = item.find('h2', {'class' : 'property-card__header'})
    nomesBairros = bairrosInformacoes.find('span', {'class' : 'property-card__address'}).text.strip()
    nomesSplit1 = nomesBairros.split('-')
    nomeSplit2 = nomesSplit1[1].split(',')
    bairros.append(nomeSplit2[0])

    # Localizando o tamanho do apartamento
    informacoesAP.append(item.find('ul', {'class' : 'property-card__details'}))
    apTamanho = item.find('span', {'class' : 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area'}).text.strip().replace('--', '0').split('-')
    tamanho.append(apTamanho[0])
    
    # Localizando os detalhes do apartamento
    outras_informacoes = item.find_all('span', {'class' : 'property-card__detail-value js-property-card-value'})
    quartos.append(outras_informacoes[0].text.strip().replace('--', '0'))
    banheiros.append(outras_informacoes[1].text.strip().replace('--', '0'))
    vagas.append(outras_informacoes[2].text.strip().replace('--', '0'))




In [35]:
dados = list(zip(tamanho, quartos, banheiros, vagas, bairros, valor))

In [36]:
dataFrame = pd.DataFrame(dados, columns=["tamanho", "quartos", "banheiros", "vagas", "bairros", "valor"])

In [37]:
def cat2int(column):
    vals = list(set(column))
    for i, string in enumerate(column):
        column[i] = vals.index(string)

In [38]:
def float2cat(column):
    for i, valor in enumerate(column):
        if valor < 500:
            column[i] = "Baixo"
        elif valor < 1000:
            column[i] = "Medio"
        else:
            column[i] = "Alto"

In [39]:
cat2int(dataFrame['bairros']) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [40]:
float2cat(dataFrame['valor'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [41]:
dataFrame

,tamanho,quartos,banheiros,vagas,bairros,valor
0,48,2,1,0,12,Baixo
1,98,2,3,1,6,Medio
2,60,2,2,1,8,Baixo
3,90,3,2,2,0,Medio
4,85,3,2,1,14,Medio
...,...,...,...,...,...,...
2395,80,2,2,1,13,Medio
2396,25,1,1,0,20,Medio
2397,83,3,2,2,6,Medio
2398,87,3,3,2,18,Medio


Questão 2 (2,5) - Treinando o modelo com os dados do site vivareal.com.br

In [42]:
X = dataFrame[['tamanho','quartos','banheiros','vagas','bairros']]
y = dataFrame.valor

Separando os dados para treinamento e teste

In [43]:
from sklearn.model_selection import train_test_split
X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X,y, test_size = 0.33)

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_data_train = sc.fit_transform(X_data_train)
X_data_test = sc.transform(X_data_test)

Treinando o modelo

In [45]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_data_train, y_data_train)

GaussianNB()

Fazendo as predições

In [47]:
y_pred = classifier.predict(X_data_test) 
y_pred

array(['Baixo', 'Baixo', 'Medio', 'Baixo', 'Medio', 'Baixo', 'Baixo',
       'Medio', 'Medio', 'Medio', 'Baixo', 'Medio', 'Medio', 'Baixo',
       'Baixo', 'Baixo', 'Medio', 'Medio', 'Medio', 'Medio', 'Baixo',
       'Baixo', 'Medio', 'Baixo', 'Medio', 'Baixo', 'Medio', 'Baixo',
       'Baixo', 'Medio', 'Medio', 'Baixo', 'Medio', 'Baixo', 'Medio',
       'Baixo', 'Medio', 'Baixo', 'Medio', 'Baixo', 'Baixo', 'Baixo',
       'Medio', 'Baixo', 'Baixo', 'Baixo', 'Medio', 'Baixo', 'Medio',
       'Baixo', 'Baixo', 'Baixo', 'Medio', 'Baixo', 'Medio', 'Medio',
       'Medio', 'Baixo', 'Medio', 'Baixo', 'Medio', 'Medio', 'Medio',
       'Baixo', 'Baixo', 'Medio', 'Medio', 'Baixo', 'Baixo', 'Baixo',
       'Medio', 'Medio', 'Medio', 'Baixo', 'Medio', 'Baixo', 'Medio',
       'Baixo', 'Baixo', 'Medio', 'Medio', 'Baixo', 'Medio', 'Baixo',
       'Medio', 'Baixo', 'Medio', 'Medio', 'Baixo', 'Medio', 'Medio',
       'Baixo', 'Medio', 'Medio', 'Medio', 'Medio', 'Medio', 'Baixo',
       'Baixo', 'Med

Exibindo matriz de precisão/Acuracia/Precision/Recall/F1-Score

In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_data_test, y_pred)
from sklearn.metrics import accuracy_score 
print ("Accuracy : ", accuracy_score(y_data_test, y_pred))
from sklearn.metrics import precision_recall_fscore_support 
print ("Precision : ", precision_recall_fscore_support(y_data_test, y_pred, average='weighted'))
from sklearn.metrics import recall_score 
print ("Recall : ", recall_score(y_data_test, y_pred, average='weighted'))
from sklearn.metrics import f1_score 
print ("F1 : ", f1_score(y_data_test, y_pred, average='weighted'))

Accuracy :  0.8661616161616161
Precision :  (0.8686483875682168, 0.8661616161616161, 0.8666072633385029, None)
Recall :  0.8661616161616161
F1 :  0.8666072633385029
